In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Clustering_codeAlong').getOrCreate()

In [4]:
data = spark.read.csv('seeds_dataset.csv', header=True,
                     inferSchema=True)

In [5]:
data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [6]:
data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [7]:
from pyspark.ml.clustering import KMeans

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [11]:
assembler = VectorAssembler(inputCols=data.columns, outputCol='features')

In [12]:
final_data = assembler.transform(data)

In [14]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
from pyspark.ml.feature import StandardScaler

In [16]:
#check with shit + Tab to know more about the standard Scaler
scaler = StandardScaler(inputCol='features',outputCol='scaledFeatures')

In [17]:
scaler_model = scaler.fit(final_data)

In [18]:
final_data = scaler_model.transform(final_data)

In [20]:
final_data.head()

Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))

In [21]:
kmeans = KMeans(featuresCol='scaledFeatures',k=3)

In [22]:
model = kmeans.fit(final_data)

In [23]:
#look for the centers
centers = model.clusterCenters()

In [24]:
print(centers)

[array([ 4.87257659, 10.88120146, 37.27692543, 12.3410157 ,  8.55443412,
        1.81649011, 10.32998598]), array([ 6.31670546, 12.37109759, 37.39491396, 13.91155062,  9.748067  ,
        2.39849968, 12.2661748 ]), array([ 4.06105916, 10.13979506, 35.80536984, 11.82133095,  7.50395937,
        3.27184732, 10.42126018])]


In [26]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
+----------+
only showing top 20 rows

